For exporting to PDF:
```bash
apt-get install pandoc 
```

Install dependencies:
```bash
pip install bokeh
pip install jupyter_bokeh
```


In [160]:

import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import re

from numpy import NaN

results = "./full-stack-test/scorer_load_test_1500_15min.csv"
df = pd.read_csv(results, low_memory=False)
if "tracked=true" in df.extra_tags.unique():
    df = df[df["extra_tags"]=="tracked=true"]
df["timestamp"] = pd.to_datetime(df.timestamp, unit='s')

output_notebook()

df

Loading BokehJS ...

,metric_name,timestamp,metric_value,check,error,error_code,expected_response,group,method,name,proto,scenario,service,status,subproto,tls_version,url,extra_tags,metadata
0,vus,2024-05-10 12:50:45,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,vus_max,2024-05-10 12:50:45,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,vus,2024-05-10 12:50:46,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vus_max,2024-05-10 12:50:46,55.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vus,2024-05-10 12:50:47,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516947,vus,2024-05-10 13:07:01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1516948,vus_max,2024-05-10 13:07:01,500.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1516949,data_sent,2024-05-10 13:07:01,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1516950,data_received,2024-05-10 13:07:01,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:

def extract_url_prefix(url):
    if url is NaN or 'localhost' in url:
        return None
    pattern = r"(.*)(0x[a-fA-F0-9]{40})"
    match = re.match(pattern, url)
    return match.group(1) if match else url

df['normalized_url'] = df['url'].apply(extract_url_prefix)
df['normalized_url']

df = df[df['normalized_url'].notna()]

In [162]:
df.metric_name.unique()

array(['http_reqs', 'http_req_duration', 'http_req_blocked',
       'http_req_connecting', 'http_req_tls_handshaking',
       'http_req_sending', 'http_req_waiting', 'http_req_receiving',
       'http_req_failed'], dtype=object)

In [163]:
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [164]:
df[df.metric_name == "http_reqs"].groupby(by=["status", "normalized_url", "method"]).count()

metric_name  \
status normalized_url                                     method                
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET            6935   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET           26777   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET           26908   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... POST           6958   
401.0  https://api.staging.scorer.gitcoin.co/passport-... GET           26849   
403.0  https://api.staging.scorer.gitcoin.co/ceramic-c... GET           27076   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH         27220   
502.0  https://api.staging.scorer.gitcoin.co/account/n... GET              23   
       https://api.staging.scorer.gitcoin.co/passport-... GET              59   

                                                                  timestamp  \
status normalized_url                                     method              
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET          6935   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         26777   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         26908   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... POST         6958   
401.0  https://api.staging.scorer.gitcoin.co/passport-... GET         26849   
403.0  https://api.staging.scorer.gitcoin.co/ceramic-c... GET         27076   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH       27220   
502.0  https://api.staging.scorer.gitcoin.co/account/n... GET            23   
       https://api.staging.scorer.gitcoin.co/passport-... GET            59   

                                                                  metric_value  \
status normalized_url                                     method                 
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET             6935   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            26777   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            26908   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... POST            6958   
401.0  https://api.staging.scorer.gitcoin.co/passport-... GET            26849   
403.0  https://api.staging.scorer.gitcoin.co/ceramic-c... GET            27076   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH          27220   
502.0  https://api.staging.scorer.gitcoin.co/account/n... GET               23   
       https://api.staging.scorer.gitcoin.co/passport-... GET               59   

                                                                  check  \
status normalized_url                                     method          
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET         0   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         0   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         0   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... POST        0   
401.0  https://api.staging.scorer.gitcoin.co/passport-... GET         0   
403.0  https://api.staging.scorer.gitcoin.co/ceramic-c... GET         0   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH       0   
502.0  https://api.staging.scorer.gitcoin.co/account/n... GET         0   
       https://api.staging.scorer.gitcoin.co/passport-... GET         0   

                                                                  error  \
status normalized_url                                     method          
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET         0   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         0   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         0   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... POST        0   
401.0  https://api.staging.scorer.gitcoin.co/passport-... GET         0   
403.0  https://api.staging.scorer.gitcoin.co/ceram

In [165]:
# df['normalized_url'] = df.url.str.extract(r'(?<=http://localhost:8080)(.*)')
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [166]:
df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    # "slate gray",
    # "forest green",
    "lavender",
    "coral",
    "turquoise",
    "gold"
]


color = iter(colors)
p = figure(title="Duration (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        # p.line(group.index, group["metric_value"]["min"], legend_label=f"min - {url}", line_width=2, color=next(color))
        # p.line(group.index, group["metric_value"]["max"], legend_label=f"max - {url}", line_width=2, color=next(color))
        p.line(group.index, group["metric_value"]["mean"], legend_label=f"mean - {url}", line_width=2, color=next(color))


show(p)


In [167]:

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]


p = figure(title="Req / second (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        print(idx)
        p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])


show(p)

0
4
6


In [168]:

from math import pi
from bokeh.models import DatetimeTickFormatter
from collections import defaultdict

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

# dfts = dft.groupby(by=["url"]).resample("1Min").agg({
#     "metric_value": ["min", "max", "mean", "count"]
# })

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]

p = figure(title="Req / second (status != 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

p.xaxis.major_label_orientation = pi/4

# Initialize dictionaries for tracking sums per URL
url_200_sums = defaultdict(int)
url_non_200_sums = defaultdict(int)

# Loop through the grouped data and collect sums for each URL
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    # Sum up counts for each status
    total_count = group["metric_value"]["count"].sum()

    # Were unable to get 200 for auth step so expecting it to return a 400
    if url == 'https://api.staging.scorer.gitcoin.co/ceramic-cache/authenticate' and status == 400:
        url_200_sums[url] += total_count

    
    # Check if the status is 200 or not and add to the appropriate sum
    if int(status) == 200:
        url_200_sums[url] += total_count
    else:
        url_non_200_sums[url] += total_count

# Print summary for each individual URL
for url in url_200_sums.keys() | url_non_200_sums.keys():
    sum_200 = url_200_sums[url]
    sum_non_200 = url_non_200_sums[url]
    total = sum_200 + sum_non_200
    success_rate = (float(sum_200) / total if total > 0 else 0) * 100

    print(f"URL: {url}")
    print(f"   200       {sum_200}")
    print(f"   non-200   {sum_non_200}")
    print(f"   success {success_rate:.2f}%\n")

# Optional: If you also want to show the plots
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    print(idx)
    p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])

show(p)

URL: https://api.staging.scorer.gitcoin.co/account/nonce
   200       6935
   non-200   23
   success 99.67%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/score/
   200       0
   non-200   27076
   success 0.00%

URL: https://api.staging.scorer.gitcoin.co/passport-admin/banners
   200       0
   non-200   26908
   success 0.00%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/stamps/bulk
   200       0
   non-200   27220
   success 0.00%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/weights
   200       26908
   non-200   0
   success 100.00%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/authenticate
   200       6958
   non-200   6958
   success 50.00%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/stamp?address=
   200       26777
   non-200   0
   success 100.00%

0
1
2
3
4
5
6
7
8
